In [90]:
import pymongo
import os
from dotenv import load_dotenv
import exam_settings
import importlib
from xhtml2pdf import pisa
import sys
#import cv2
# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

importlib.reload(exam_settings)

import const
#importlib.reload(const)

In [55]:
load_dotenv() 

db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']
collection = db['tb_aws_sa']


In [56]:
IMG_FOLDER = 'aws_sa_data/img_youtube'
PDF_FOLDER = 'aws_sa_data/pdf/'


In [15]:
#get question(s) and export to images
def get_n_export_question_2_img(question_num):
    pipeline = [
                {"$match": {'exported': 0, 'type': 'multiple-choice'}},
                {"$sample": {"size": question_num}} #randomly
            ]
    random_documents = list(collection.aggregate(pipeline))
    index = 1
    for doc in random_documents:
        str_index = str(index) + ') '
        #question first
        html_question = '<div class="question">'+str_index + doc['question']+'</div>'
        html_answers_start = '<div class="answers">'
        html_answers_end = '</div>'
        #First image: options without explanations
        for key in doc['options'].keys():
            html_answers_start += f'''
                <div class="answer">
                    <label>{key}. {doc['options'][key]}</label>
                </div>
                <div class="explanation">
                    <label>{key}. {doc['explanation'][key]}</label>
                </div>'''
        #1 doc 1 image
        exam_settings.generate_image(exam_settings.html_head_str + html_question + html_answers_start + html_answers_end + exam_settings.html_tail_str, IMG_FOLDER, doc['uuid'] + '.png')
        #Second image: options with explanations
        html_answers_start = '<div class="answers">'
        for key in doc['options'].keys():
            correct_class = ''
            if doc['answer'] == key:
                correct_class = ' correct'
            html_answers_start += f'''
                <div class="answer">
                    <label class="{correct_class}">{key}. {doc['options'][key]}</label>
                </div>
                <div class="explanation show">
                    <label>{doc['explanation'][key]}</label>
                </div>'''
        exam_settings.generate_image(exam_settings.html_head_str + html_question + html_answers_start + html_answers_end + exam_settings.html_tail_str, IMG_FOLDER, doc['uuid'] + '_explain.png')
        #
        index += 1
    # print(random_documents)

#test
#get_n_export_question_2_img(2)

In [ ]:
def export_2_videos():
    images = sorted([f for f in os.listdir(IMG_FOLDER) if f.endswith(('.png'))])
    if not images:
        print(f"No images found in the folder: {IMG_FOLDER}")
        return
    #print(images)
    first_image_path = os.path.join(IMG_FOLDER, images[0])
    
    try:
        img = cv2.imread(first_image_path)
        height, width, channels = img.shape
    except Exception as e:
        print(f"Error reading the first image: {e}")
        return
    fps=1/60    #1 frame for 60 seconds
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec if needed (e.g., 'XVID', 'MJPG')
    out = cv2.VideoWriter('abc.mp4', fourcc, fps, (width, height))

    print("Creating video...")
    for image_name in images:
        image_path = os.path.join(IMG_FOLDER, image_name)
        try:
            img = cv2.imread(image_path)
            if img is None:
                print(f"Error reading image: {image_name}")
                continue
            out.write(img)
            print(image_path)
        except Exception as e:
            print(f"Error processing image {image_name}: {e}")

    out.release()
    # Closes all the frames
    cv2.destroyAllWindows()
    print(f"Video saved to: {VID_FOLDER}")

#test
#export_2_videos()


Creating video...
aws_cloud_practitioner/img/309eb2c9-5ab1-44cd-af9a-5351a678407e.png
aws_cloud_practitioner/img/309eb2c9-5ab1-44cd-af9a-5351a678407e_explain.png
aws_cloud_practitioner/img/d1cca612-c6ed-43a3-b8c9-def68edd4da6.png
aws_cloud_practitioner/img/d1cca612-c6ed-43a3-b8c9-def68edd4da6_explain.png
aws_cloud_practitioner/img/d556280b-72ee-4e9e-b69f-6182b3981f8e.png
aws_cloud_practitioner/img/d556280b-72ee-4e9e-b69f-6182b3981f8e_explain.png
aws_cloud_practitioner/img/e2151b3e-83cd-435d-b7a1-b5ac13166eaa.png
aws_cloud_practitioner/img/e2151b3e-83cd-435d-b7a1-b5ac13166eaa_explain.png
Video saved to: aws_cloud_practitioner/vid


In [94]:
#get question(s) and export to images for PDF file (5 pages, 5 questions per page)
def get_n_export_questions_2_pdf(cert_name, filename, page_num, question_per_page, landing_url):
    pipeline = [
                {"$match": {'img_exported': None}},
                {"$sample": {"size": question_per_page * page_num}} #randomly
            ]
    random_documents = list(collection.aggregate(pipeline))
    question_index = 1
    #print(random_documents)
    html_question_list = []
    #1. Question pages
    for page_idx in range(page_num):
        #begin page
        html_question_list.append('<div class="container">')
        for question_idx in range(question_per_page):
            str_index = str(question_index) + ') '
            current_doc = random_documents[question_index-1]
            html_question = '<div class="question">'+str_index + current_doc['question']+'</div>'
            html_answers_start = '<div class="answers">'
            html_answers_end = '</div>'
            for key in current_doc['options'].keys():
                html_answers_start += f'''
                    <div class="answer">
                        <label>{key}. {current_doc['options'][key]}</label>
                    </div>'''
            html_question_list.append(html_question + html_answers_start + html_answers_end)    #1 question & options
            question_index += 1
        #end 1 page
        html_question_list.append(f'<div class="checkout">{cert_name}<br/>Check out more questions <a href={landing_url}">{landing_url}</a></div>')
        html_question_list.append('</div><pdf:nextpage />') #close tag of 1 page
    #2. Answers
    question_index = 1
    for page_idx in range(page_num):
        #begin page
        html_question_list.append('<div class="container">')
        for question_idx in range(question_per_page):
            str_index = str(question_index) + ') '
            current_doc = random_documents[question_index-1]
            html_question = '<div class="question">'+str_index + current_doc['question']+'</div>'
            html_answers_start = '<div class="answers">'
            html_answers_end = '</div>'
            for key in current_doc['options'].keys():
                correct_class = ''
                if key == current_doc['answer']:
                    correct_class = 'correct'
                html_answers_start += f'''
                    <div class="explanation show">
                        <label class="{correct_class}">{key}. {current_doc['explanation'][key]}</label>
                    </div>'''
            html_question_list.append(html_question + html_answers_start + html_answers_end)    #1 question & options
            question_index += 1
        #end 1 page
        #html_question_list.append(f'<div class="checkout">{cert_name}<br/>Check out more questions <a href={landing_url}">{landing_url}</a></div>')
        html_question_list.append('</div>')
    #combine into a string
    html_question_list_str = ('').join(html_question_list)
    #export to pdf
    with open(PDF_FOLDER + filename, "w+b") as result_file:
        # convert HTML to PDF
        pisa_status = pisa.CreatePDF(
                exam_settings.html_pdf_head_str + html_question_list_str + exam_settings.html_pdf_tail_str,       # page data
                dest=result_file  # destination file
            )
        # Check for errors
        if pisa_status.err:
            print("An error occurred!")
        #

#test
current_date = const.get_current_date_yyyymmdd()
cert_name = f'AWS Certified Solutions Architect - Associate Practice Questions - {current_date}'
filename = 'aws_saa_' + current_date + '.pdf'
get_n_export_questions_2_pdf(cert_name, filename, 4, 5, 'https://www.udemy.com/course/aws-certified-solutions-architect-associate-practice-exams-iq/')